In [1]:
import sys
sys.path.append("./Bspli/")
import os  
import faiss
import time
import numpy as np 
import index 
import torch
import pandas as pd

mnist = np.load("dataset/mnist-784-euclidean.npy")
print(f'mnist data shape: {mnist.shape}')
print(f'mnist dtype: {mnist.dtype}')

# mnist = np.random.uniform(low=0, high=255, size=(100000, 2))
# mnist = mnist.astype(np.float32)
# print(f'mnist dtype: {mnist.dtype}')
# print(f'mnist data shape: {mnist.shape}')


mnist data shape: (60000, 784)
mnist dtype: float32


In [2]:
%%time
# Brute Search
flat = faiss.IndexFlatL2(mnist.shape[1])
flat.add(mnist)

D, FLAT_I = flat.search(mnist[0].reshape(1, mnist.shape[1]), k=100) 
print(f'brute query: {FLAT_I}')

brute query: [[    0 32248  8728 18932 30483 24149 42338 52295 26251 50173 53634 24330
  54159 57528  1482 53428 18123 31379 52864 10536 29719 36087 30489 23947
  20034 52057 33825 21654 31008 55208 22477 44282 47968 54203 19825  1634
  27378 33909 15378 24708 34474 26413 16017 46824 46358  1516 34557 16832
  21629 29021 10740 24107  5688 52665  1864  5036 39031  1978 40546 22322
  52231 37284 24730  5970 21976 16945  9568 36697 25675 54189 11396 42555
  33445 52540 44263 18404 19186 24232 54184 25762 14736 33970  5210 59212
   8642 22569 15052  2933  6772 22963  6516   832 21244 21583 35838 59846
  21210 13502 52559 13862]]
CPU times: total: 62.5 ms
Wall time: 78.8 ms


In [3]:
mnist_tensor = torch.from_numpy(mnist)
print(f'mnist tensor shape: {mnist_tensor.shape}')

idx = index.Indexing(
    gl_size=10000, 
    ll_size=3000,
    g_epoch_num=5,
    l_epoch_num=10,
    g_hidden_size=10,
    l_hidden_size=20,
    g_block_range=4,
    l_block_range=4,
    random_partitioning=False
)
idx.train(mnist_tensor)

print(f"local model len:{len(idx._l_model)}")

mnist tensor shape: torch.Size([60000, 784])
torch.Size([1995, 785])
torch.Size([8906, 785])
torch.Size([4610, 785])
torch.Size([3738, 785])
torch.Size([7329, 785])
torch.Size([4915, 785])
torch.Size([8447, 785])
torch.Size([4660, 785])
torch.Size([2740, 785])
torch.Size([2620, 785])
torch.Size([382, 785])
torch.Size([9658, 785])
first stage partitioning finish
partitioning blocks : 12
training local model
1, 10 loss: -0.0
2, 10 loss: -0.0
3, 10 loss: -0.0
4, 10 loss: -0.0


d:\conda\envs\ml\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


5, 10 loss: -0.0
6, 10 loss: -0.0
7, 10 loss: -0.0
8, 10 loss: -0.0
9, 10 loss: -0.0
10, 10 loss: -0.0
training local model
1, 10 loss: 0.008314797878265381
1, 20 loss: 0.007823784351348877
1, 30 loss: 0.00731235682964325
1, 40 loss: 0.007242535948753357
2, 10 loss: 0.006758679747581482
2, 20 loss: 0.006528628468513489
2, 30 loss: 0.006562437415122985
2, 40 loss: 0.0066947627067565916
3, 10 loss: 0.006260637044906616
3, 20 loss: 0.006269686818122864
3, 30 loss: 0.006140231490135193
3, 40 loss: 0.006330084204673767
4, 10 loss: 0.0061553555727005
4, 20 loss: 0.006210472583770752
4, 30 loss: 0.0061468368768692015
4, 40 loss: 0.006129461526870728
5, 10 loss: 0.006028050184249878
5, 20 loss: 0.005976652503013611
5, 30 loss: 0.006014922261238098
5, 40 loss: 0.005825188159942627
6, 10 loss: 0.0058271831274032595
6, 20 loss: 0.0059806954860687255
6, 30 loss: 0.005940693020820617
6, 40 loss: 0.005667051672935486
7, 10 loss: 0.005983898043632507
7, 20 loss: 0.005835117101669312
7, 30 loss: 0.005

d:\study\bspli\./Bspli\utils\Partitioning.py:111: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ma = torch.tensor(means)
d:\conda\envs\ml\lib\site-packages\torch\nn\modules\loss.py:928: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)
d:\conda\envs\ml\lib\site-packages\torch\nn\modules\loss.py:928: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


In [4]:
%%time

def recall(pred, true):
    x = np.isin(pred, true)
    return x.sum() / true.size

qp = torch.from_numpy(mnist[0])
# print(qp)
pred = idx.query(qp, k=100)
print(f"recall: {recall(pred, FLAT_I)}")

pred = pred.to(torch.int)
print(f"pred: {pred}")

print(f"isin same block: {idx.isin(qp)}")

recall: 0.7
pred: tensor([    0, 32248,  8728, 18932, 24149, 42338, 52295, 26251, 50173, 53634,
        24330, 54159, 57528,  1482, 18123, 23947, 20034, 52057, 33825, 21654,
        31008, 55208, 22477, 44282, 47968, 54203, 19825, 33909, 24708, 34474,
        26413, 16017, 46824, 46358,  1516, 34557, 21629, 29021, 10740, 24107,
        52665,  1864,  5036, 40546, 22322, 52231, 37284, 21976, 36697, 25675,
        54189, 11396, 42555, 52540, 44263, 18404, 25762, 14736,  5210, 59212,
        22569,  6772, 21244, 21583, 35838, 59846, 21210, 13502, 52559, 13862,
        18172, 41980, 43997, 53812, 18162, 15975, 50187, 28384, 35570, 24271,
        21661, 15907, 38896, 15444, 50071, 45567, 26313,  2395, 11936, 16112,
        49334, 44476, 18189, 17298, 29023, 26364, 36242, 18197, 38940, 25557],
       dtype=torch.int32)
isin same block: False
CPU times: total: 625 ms
Wall time: 100 ms


In [5]:
%%time
pred = idx.query_with_threshold(qp, k=100, threshold=0.02, g_block_range=10)
print(f"recall: {recall(pred, FLAT_I)}")
pred = pred.to(torch.int)
print(f"pred: {pred}")

recall: 0.95
pred: tensor([    0, 32248,  8728, 18932, 30483, 24149, 42338, 52295, 26251, 50173,
        53634, 24330, 54159, 57528,  1482, 53428, 18123, 31379, 52864, 10536,
        36087, 30489, 23947, 20034, 52057, 33825, 21654, 31008, 55208, 22477,
        44282, 47968, 54203, 19825,  1634, 27378, 33909, 15378, 24708, 34474,
        26413, 16017, 46824, 46358,  1516, 34557, 21629, 29021, 10740, 24107,
         5688, 52665,  1864,  5036, 39031,  1978, 40546, 22322, 52231, 37284,
         5970, 21976, 16945, 36697, 25675, 54189, 11396, 42555, 33445, 52540,
        44263, 18404, 24232, 54184, 25762, 14736, 33970,  5210, 59212,  8642,
        22569, 15052,  2933,  6772, 22963,  6516,   832, 21244, 21583, 35838,
        59846, 21210, 13502, 52559, 13862, 18172, 41980, 43997, 53812, 18162],
       dtype=torch.int32)
CPU times: total: 656 ms
Wall time: 116 ms


In [6]:
result = []

def benchmark_knn_query(data, index, size=1000, k=100):
    indices = np.random.choice(data.shape[0], size, replace=False)
    query_time = 0
    cur_recall = 0
    isin_count = 0

    # query
    for i in indices:
        q = torch.from_numpy(data[i])
        start = time.time()
        qk = index.query(q, k=100, g_block_range=7, l_block_range=4)
        query_time += (time.time() - start)
        D, FLAT_I = flat.search(data[i].reshape(1, data.shape[1]), k=k) 
        cur_recall += recall(qk, FLAT_I)
        # isin_count += 1 if index.isin(q) else 0
        
    result.append((query_time/1000, cur_recall/1000, isin_count/1000))

In [7]:
%%script false --no-raise-error
# recall(pred, FLAT_I)

benchmark_knn_query(mnist, idx, size=1000, k=100)
print(result)
print(idx.get_search_blocks_num())

In [8]:
result = []

def benchmark_knn_query_with_threshold(data, index, size=1000, k=100):
    indices = np.random.choice(data.shape[0], size, replace=False)
    query_time = 0
    cur_recall = 0
    isin_count = 0

    # query
    for i in indices:
        q = torch.from_numpy(data[i])
        start = time.time()
        qk = idx.query_with_threshold(q, k=100, threshold=0.05, g_block_range=12)
        query_time += (time.time() - start)
        D, FLAT_I = flat.search(data[i].reshape(1, data.shape[1]), k=k) 
        cur_recall += recall(qk, FLAT_I)
        # isin_count += 1 if index.isin(q) else 0
        
    result.append((query_time/1000, cur_recall/1000, isin_count/1000))



In [9]:
%%script false --no-raise-error
benchmark_knn_query_with_threshold(mnist, idx, size=1000, k=100)

print(result)